In [1]:
from drn_interactions.io import load_derived_generic
from drn_interactions.transforms import SpikesHandler
from drn_interactions.surrogates import shuffle_spikes
from drn_interactions.transforms.brain_state import StateHandler, RawEEGHandler
from drn_interactions.transforms.brain_state_spikes import (
    align_spikes_to_states_long, align_spikes_to_phase_long, align_bins_to_states_long,
    )
from drn_interactions.responders.brain_state_responders import BSResonders
from drn_interactions.transforms.nbox_transforms import segment_spikes
from drn_interactions.spiketrains.spiketrain_stats import cv_isi_burst
from drn_interactions.config import ExperimentInfo, Config
from drn_interactions.responders.brain_state import SpikeRateResonders
from drn_interactions.responders.brain_state import PhaseLockResponders
from drn_interactions.spiketrains.neurontype_props import ChiSquarePostHoc

from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
import numpy as np
import pandas as pd
sns.set_theme(context="poster", style="ticks")
from drn_interactions.plots.circular import circular_hist

from IPython.display import display

%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

c:\Users\roryl\.conda\envs\drn38\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
c:\Users\roryl\.conda\envs\drn38\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


# Spike Rate Change During EEG States


### Load Data

In [3]:
session_names = ExperimentInfo.eeg_sessions

neuron_types = load_derived_generic("neuron_types.csv")
states_handler = StateHandler(
    quality_to_include=("good",),
    t_start=0,
    t_stop=1800,
    session_names=session_names,
)
spikes_handler = SpikesHandler(
    block="pre",
    t_start=0,
    bin_width=1,
    t_stop=1800,
    session_names=session_names,
    
)

df_aligned = align_bins_to_states_long(
    spikes_handler=spikes_handler,
    states_handler=states_handler,
    neuron_types=neuron_types
)
df_aligned["zcounts"] = (
    df_aligned
    .groupby("neuron_id")["counts"]
    .transform(zscore)
)

### Calculate Responders

- Mixed ANOVA for interactions within neurons (brain states) and among neurons (neuron types)
- Post hoc responder status for each neuron using Mann-Whitney U test 

In [3]:

mod = SpikeRateResonders(df_value_col="zcounts", round_output=2)
anova, contrasts = mod.get_anova(df_aligned, fit_neuron_types=True)

display(anova)
# display(contrasts)

responders = mod.get_responders(df_aligned, abs_diff_thresh=0.1)
display(responders.sample(3))

responders.to_csv(
    Config.derived_data_dir / "brain_states_spikerate_responders.csv",
    index=False,
)

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,neuron_type,0.04,2,312,0.02,0.54,0.58,0.00,NaN
1,state,4.84,1,312,4.84,18.76,0.00,0.06,1.0
2,Interaction,0.18,2,312,0.09,0.36,0.70,0.00,NaN


# Phase Locking Analysis

### Load Data and Align to EEG Oscillation Phase
- Raw EEG signal downsampled to 250 Hz
- In activated brain states, it is filtered between 4 - 8 Hz
- In slow wave states, it is filtered between 0.5 - 4 Hz
- Spike times are aligned to EEG phase separately for each brain state
- In each state, the distrobution of phases is tested for uniformity using Rayleigh tests
- The prefered phase of each neuron and whether it is significantly different phase locked is saved in a file
- This file is loaded into R and analysed using an GLM on angular embeddings (see below)

In [13]:
spikes_handler = SpikesHandler(
    block="pre",
    t_start=0,
    bin_width=1,
    t_stop=1800,
    session_names=session_names,
)

eeg_handler = RawEEGHandler(
    block="pre",
    t_start=0,
    t_stop=1800,
    session_names=session_names,
)
df_aligned_phase = align_spikes_to_phase_long(
    spikes_handler=spikes_handler,
    states_handler=states_handler,
    raw_eeg_handler=eeg_handler,
    neuron_types=neuron_types,
).dropna()


df_sw = df_aligned_phase.query("state == 'sw'")
df_act = df_aligned_phase.query("state == 'act'")

mod = PhaseLockResponders(round_output=2, fs=(250 * 6) / (2 * np.pi))
df_res_act = mod.prefered_angles(df_act, phase_col="theta_phase")
df_res_sw = mod.prefered_angles(df_sw, phase_col="delta_phase",)
df_prefered_angles = pd.concat([(
        df_res_sw
        .assign(oscillation="delta")
        [["neuron_id", "oscillation", "mean_angle", "p"]]
        ),
        (
            df_res_act
            .assign(oscillation="theta")
            [["neuron_id", "oscillation", "mean_angle", "p"]]
        )
]
)
df_prefered_angles = df_prefered_angles.merge(neuron_types)
display(df_prefered_angles.sample(3))
df_prefered_angles.to_csv(Config.derived_data_dir / "brain_states_phase_responders.csv", index=False)

c:\users\roryl\repos\drn interactions\drn_interactions\transforms\brain_state_spikes.py:94: FutureWarning: Passing 'suffixes' which cause duplicate columns {'group_name_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_aligned = df_aligned.merge(


,neuron_id,oscillation,mean_angle,p,neuron_type,session_name,group_name
422,2414,delta,-0.40,0.01,SR,acute_16,acute_saline
373,2329,theta,1.66,0.05,SIR,acute_15,acute_saline
409,2406,theta,1.59,0.00,SR,acute_16,acute_saline


In [14]:
df_prefered_angles = df_prefered_angles.assign(sig=lambda x: x.p < 0.05)
mod = ChiSquarePostHoc(value_col="sig", round=2)

display(mod(df_prefered_angles))
display(mod(df_prefered_angles.query("oscillation == 'delta'")))
display(mod(df_prefered_angles.query("oscillation == 'theta'")))

anova                         Chi2(2)=32.0 (p=0.00*)
SIR - FF     41.4%; 72.39% | Chi(1.0)=30.62 (p=0.0*)
SIR - SR      41.4%; 54.78% | Chi(1.0)=8.08 (p=0.0*)
FF - SR     72.39%; 54.78% | Chi(1.0)=10.93 (p=0.0*)
dtype: object

anova                         Chi2(2)=16.4 (p=0.00*)
SIR - FF    57.94%; 86.76% | Chi(1.0)=14.86 (p=0.0*)
SIR - SR     57.94%; 64.23% | Chi(1.0)=0.76 (p=0.38)
FF - SR     86.76%; 64.23% | Chi(1.0)=10.29 (p=0.0*)
dtype: object

anova                        Chi2(2)=19.9 (p=0.00*)
SIR - FF    25.0%; 57.58% | Chi(1.0)=17.21 (p=0.0*)
SIR - SR     25.0%; 45.19% | Chi(1.0)=9.73 (p=0.0*)
FF - SR     57.58%; 45.19% | Chi(1.0)=2.25 (p=0.13)
dtype: object

### Group Level Statistics

Here we use R to compute the group level statistics. We use bayesian ANOVAs on angular embeddings using the `bpnreg` package. 

See [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6218623/) for a discription of the approach and the packages [documentation here](https://cran.r-project.org/web/packages/bpnreg/bpnreg.pdf). 

In [15]:
%%R
suppressMessages(require(circular))
suppressMessages(require(bpnreg))
suppressMessages(require(tidyverse))

p <- "C:\\Users\\roryl\\repos\\DRN Interactions\\data\\derived\\brain_states_phase_responders.csv"

df <- suppressMessages(read_csv(p)) %>%
  mutate(
    ang = circular(mean_angle, units="radians"),
    oscillation=factor(oscillation, levels=c("delta", "theta")),
    neuron_type=factor(neuron_type, levels=c("SR", "SIR", "FF"))
    )


In [16]:
%%R
# summary stats for delta
df %>%
  filter(oscillation=="delta", p < 0.05) %>%
  group_by(neuron_type) %>%
  summarise(
    theta=circular::mean.circular(ang),
    var=circular::var.circular(ang),
    rho=circular::rho.circular(ang)
    ) %>% print

# summary stats for theta
df %>%
  filter(oscillation=="theta", p < 0.05) %>%
  group_by(neuron_type) %>%
  summarise(
    theta=circular::mean.circular(ang),
    var=circular::var.circular(ang),
    rho=circular::rho.circular(ang)
    ) %>% print

# A tibble: 3 x 4
  neuron_type theta        var   rho
  <fct>       <circular> <dbl> <dbl>
1 SR          -1.951247  0.702 0.298
2 SIR         -1.560890  0.848 0.152
3 FF          -2.431734  0.556 0.444
# A tibble: 3 x 4
  neuron_type theta        var   rho
  <fct>       <circular> <dbl> <dbl>
1 SR          1.842788   0.562 0.438
2 SIR         1.902326   0.714 0.286
3 FF          1.750418   0.465 0.535


In [17]:
%%R

invisible(capture.output(
  mod_delta <- bpnreg::bpnr(
  ang ~ neuron_type,
  data=filter(df, oscillation=="delta", p < 0.05)
  )
))
  
coef_delta <- bpnreg::coef_circ(
  mod_delta, type="categorical", units="radians"
  )
print(round(coef_delta$Means, 2))
print(round(coef_delta$Differences, 2))

                             mean  mode   sd    LB    UB
(Intercept)                 -2.03 -2.02 0.26 -2.56 -1.51
neuron_typeSIR              -1.52 -1.63 0.74 -2.90  0.17
neuron_typeFF               -2.50 -2.45 0.19 -2.88 -2.18
neuron_typeSIRneuron_typeFF -2.57 -2.49 0.60  2.46  5.00
                             mean  mode   sd    LB   UB
neuron_typeSIR              -0.51 -0.75 0.78 -2.10 1.07
neuron_typeFF                0.47  0.43 0.32 -0.20 1.07
neuron_typeSIRneuron_typeFF  0.55  0.46 0.73 -1.02 1.94


In [18]:
%%R

invisible(capture.output(
  mod_theta <- bpnreg::bpnr(
    ang ~ neuron_type,
    data=filter(df, oscillation=="theta", p < 0.05)
    )
))
coef_theta <- bpnreg::coef_circ(
  mod_theta, type="categorical", units="radians"
  )
print(round(coef_theta$Means, 2))
print(round(coef_theta$Differences, 2))

                            mean mode   sd   LB   UB
(Intercept)                 1.81 1.76 0.18 1.49 2.16
neuron_typeSIR              1.87 1.88 0.54 0.86 2.97
neuron_typeFF               1.71 1.64 0.19 1.35 2.08
neuron_typeSIRneuron_typeFF 1.72 1.75 0.56 0.67 2.92
                             mean  mode   sd    LB   UB
neuron_typeSIR              -0.06 -0.04 0.57 -1.18 1.06
neuron_typeFF                0.10  0.20 0.27 -0.37 0.69
neuron_typeSIRneuron_typeFF  0.09  0.34 0.64 -1.30 1.28


In [19]:
%%R
invisible(capture.output(
  mod_oscillation <-  bpnreg::bpnr(
    ang ~ oscillation,
    data=filter(df, p < 0.05)
  )
))
coef_oscillation <- bpnreg::coef_circ(
  mod_oscillation, type="categorical", units="radians"
)
print(round(coef_oscillation$Means, 2))
print(round(coef_oscillation$Differences, 2))

                  mean  mode   sd    LB    UB
(Intercept)      -2.19 -2.26 0.17 -2.57 -1.88
oscillationtheta  1.77  1.77 0.13  1.51  2.05
                 mean mode   sd  LB   UB
oscillationtheta 2.32  2.3 0.22 1.9 2.78
